- 자생식물 데이터 병합

In [26]:
import numpy as np
import pandas as pd
import glob
import json
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import quote 
import time
import requests

In [ ]:
# json 파일이 있는 폴더에서 실행 

output = glob.glob('CR*_*_*.json')
output[1]

In [ ]:
import json
import glob

data = []
for f in glob.glob("CR*_*_*.json"):
    with open(f, encoding="utf-8") as infile:
        data.append(json.load(infile))

In [28]:
# 이름 : wd_plnt_nm, 학명 : scientific_nm, 주소 : region_nm

n = []
sn = []
pl = []

for i, dt in enumerate(data):
    name = data[i]['plants']['wd_plnt_nm']
    s_name = data[i]['plants']['scientific_nm'][:-7]
    place = data[i]['images']['region_nm']

    n.append(name)
    sn.append(s_name)
    pl.append(place)

In [21]:
# df0 = pd.DataFrame({'이름': n,'학명': sn,'주소': pl})
# df0.to_csv('자생식물1.csv')
# df0

,이름,학명,주소


In [29]:
df = pd.read_csv('자생식물1.csv')
df

,Unnamed: 0,이름,학명,주소
0,0,까마귀쪽나무,Litsea japonica (Thunb.) Juss.,한라수목원
1,1,좁은잎천선과,Ficus erecta var. sieboldii (Miq.) King,제주시 오등동
2,2,참식나무,Neolitsea sericea (Blume) Koidz.,제주특별자치도 제주시 조천읍
3,3,메밀,Fagopyrum esculentum Moench,서귀포시 안덕면 서광리
4,4,구실잣밤나무,Castanopsis sieboldii (Makino) Hatus. ex T. Ya...,만장굴
5,5,순비기나무,Vitex rotundifolia L. f.,제주특별자치도 제주시 삼양동
6,6,순비기나무,Vitex rotundifolia L. f.,제주특별자치도 제주시 구좌읍 월정리
7,7,참가시나무,Quercus salicina Blume,한라수목원
8,8,황근,Hibiscus hamabo Siebold & Zucc.,한라수목원
9,9,황근,Hibiscus hamabo Siebold & Zucc.,한라수목원


In [30]:
from urllib.parse import quote
with open('road_key.txt') as file: 
    road_key = f'{file.read()}'

In [31]:
df['주소']

0                   한라수목원
1                 제주시 오등동
2         제주특별자치도 제주시 조천읍
3            서귀포시 안덕면 서광리
4                     만장굴
5         제주특별자치도 제주시 삼양동
6     제주특별자치도 제주시 구좌읍 월정리
7                   한라수목원
8                   한라수목원
9                   한라수목원
10                  한라수목원
11                  한라수목원
Name: 주소, dtype: object

In [157]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'

ads =[]
quotess = []
urls = []
results = []
BJD_codes = []


for i in range(12):
    ad = df["주소"][i]
    ads.append(ad)
for a in ads:
    quotes = quote(a)
    quotess.append(quotes)
for q in quotess:
    params2 = f'keyword={q}&resultType=json'
    url = f"{road_url}?{params1}&{params2}"
    urls.append(url)
for u in urls:
    result = requests.get(u).json()
    results.append(result)
for r in results:
    BJD_code=r['results']['juso'][0]['admCd'] 
    BJD_codes.append(int(BJD_code))

BJD_codes

[5011013700,
 5011013400,
 5011025926,
 5013031027,
 5011025626,
 5011013000,
 5011025626,
 5011013700,
 5011013700,
 5011013700,
 5011013700,
 5011013700]

In [97]:
with open('soil_key.txt') as file:
    soilkey = file.read()
    

s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={5011013700}'
s_result = requests.get(s_url)
s_contents = s_result.text
s_soup = BeautifulSoup(s_contents, 'html.parser')

acid = []
vldpha = []
vldsia = []
om = []
posifert_mg = []
posifert_k = []
posifert_ca = []

lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca]
cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca']

for li in lis:
    for con in cons:
        if lis.index(li) == cons.index(con):
            for a in s_soup.find_all(con):
                li.append(a.get_text())

ground_con = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca})
ground_con

,산도,유효인산,유기물,마그네슘,칼륨,칼슘
0,5.5,30,81,.5,.48,.5
1,5.5,25,75,.3,.56,.3
2,5.5,22,67,.3,.44,.2
3,5.5,19,69,.3,.47,.1
4,5.5,25,60,.2,.54,.2
5,5.5,27,60,.2,.52,.1
6,5.5,25,61,.2,.49,.1
7,5.5,16,61,.2,.41,.1
8,5.5,30,62,.2,.58,.1
9,5.5,41,54,.2,.43,.1


In [156]:
for BJD_code in BJD_codes:
    s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
print(s_url)

http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey=MAW83St0u7MX4NvrMdB3L815EvMa%2Bl5IQE2mS8jMIHr49AXVHFmUbSzGp%2FqZEk3C2ckB5UcIgajRm8%2BreDnYEg%3D%3D&Page_Size=10&Page_No=1&BJD_Code=5011013700


In [163]:
s_soups = []

for BJD_code in BJD_codes:
    s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
    s_result = requests.get(s_url)
    s_contents = s_result.text
    s_soups = BeautifulSoup(s_contents, 'html.parser')

    acid = []
    vldpha = []
    om = []
    posifert_mg = []
    posifert_k = []
    posifert_ca = []
    dfs = []  

    lis = [acid, vldpha, om, posifert_mg, posifert_k, posifert_ca]
    cons = ['acid', 'vldpha', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca']
    for s in s_soups: 
        for li in lis:
            for con in cons:
                if lis.index(li) == cons.index(con):
                    for a in s_soup.find_all(con):
                        li.append(float(a.get_text()))

                        
        dfg = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca})
        dfs.appned(dfg)


AttributeError: 'list' object has no attribute 'appned'

In [150]:
l[0]

,산도,유효인산,유기물,마그네슘,칼륨,칼슘


In [73]:
with open('soil_key.txt') as file:
    soilkey = file.read()

s_soups = []

for BJD_code in BJD_codes:
    s_result = requests.get(f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}')
    s_soup = BeautifulSoup(s_result.text, 'html.parser')
    s_soups.append(s_soup)

s_soups[0]

<openapi_serviceresponse>
<cmmmsgheader>
<errmsg>SERVICE ERROR</errmsg>
<returnauthmsg>LIMITED_NUMBER_OF_SERVICE_REQUESTS_EXCEEDS_ERROR</returnauthmsg>
<returnreasoncode>22</returnreasoncode>
</cmmmsgheader>
</openapi_serviceresponse>

In [81]:
# 식물 토양 성분 
with open('soil_key.txt') as file:
    soilkey = file.read()

s_soups = []

for BJD_code in BJD_codes:
    s_result = requests.get(f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}')
    s_soup = BeautifulSoup(s_result.text, 'html.parser')
    s_soups.append(s_soup)

    acid = []
    vldpha = []
    vldsia = []
    om = []
    posifert_mg = []
    posifert_k = []
    posifert_ca = []
    dfs = []

    lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca]
    cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca']
    for s in s_soups:
        for li in lis:
            for con in cons:
                if lis.index(li) == cons.index(con):
                    for a in s.find_all(con):
                        li.append(a.get_text())

        dfg = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca})
        dfs.append(dfg)
    

In [90]:
dfs[1]

,산도,유효인산,유기물,마그네슘,칼륨,칼슘
0,5.5,30,81,.5,.48,.5
1,5.5,25,75,.3,.56,.3
2,5.5,22,67,.3,.44,.2
3,5.5,19,69,.3,.47,.1
4,5.5,25,60,.2,.54,.2
5,5.5,27,60,.2,.52,.1
6,5.5,25,61,.2,.49,.1
7,5.5,16,61,.2,.41,.1
8,5.5,30,62,.2,.58,.1
9,5.5,41,54,.2,.43,.1


In [87]:
dfs[:2]

[    산도 유효인산 유기물 마그네슘   칼륨  칼슘
 0  5.5   30  81   .5  .48  .5
 1  5.5   25  75   .3  .56  .3
 2  5.5   22  67   .3  .44  .2
 3  5.5   19  69   .3  .47  .1
 4  5.5   25  60   .2  .54  .2
 5  5.5   27  60   .2  .52  .1
 6  5.5   25  61   .2  .49  .1
 7  5.5   16  61   .2  .41  .1
 8  5.5   30  62   .2  .58  .1
 9  5.5   41  54   .2  .43  .1,
      산도   유효인산 유기물  마그네슘    칼륨    칼슘
 0   5.5     30  81    .5   .48    .5
 1   5.5     25  75    .3   .56    .3
 2   5.5     22  67    .3   .44    .2
 3   5.5     19  69    .3   .47    .1
 4   5.5     25  60    .2   .54    .2
 5   5.5     27  60    .2   .52    .1
 6   5.5     25  61    .2   .49    .1
 7   5.5     16  61    .2   .41    .1
 8   5.5     30  62    .2   .58    .1
 9   5.5     41  54    .2   .43    .1
 10  6.7     58  26   3.2   .36   6.4
 11  5.8    462  27   1.1   .21   2.5
 12  5.1    376  80     1   .48   2.5
 13  5.2    668  82    .9   .28   2.3
 14  6.4     88  21   1.1   .32   2.2
 15  6.7  441.2  58  3.56  2.56  7.28
 16    5   1

In [88]:
dfs[1:3]

[     산도   유효인산 유기물  마그네슘    칼륨    칼슘
 0   5.5     30  81    .5   .48    .5
 1   5.5     25  75    .3   .56    .3
 2   5.5     22  67    .3   .44    .2
 3   5.5     19  69    .3   .47    .1
 4   5.5     25  60    .2   .54    .2
 5   5.5     27  60    .2   .52    .1
 6   5.5     25  61    .2   .49    .1
 7   5.5     16  61    .2   .41    .1
 8   5.5     30  62    .2   .58    .1
 9   5.5     41  54    .2   .43    .1
 10  6.7     58  26   3.2   .36   6.4
 11  5.8    462  27   1.1   .21   2.5
 12  5.1    376  80     1   .48   2.5
 13  5.2    668  82    .9   .28   2.3
 14  6.4     88  21   1.1   .32   2.2
 15  6.7  441.2  58  3.56  2.56  7.28
 16    5   1014  75   1.1   .36   2.2
 17  6.4     88  21   1.1   .32   2.2
 18  6.6     10  35    .8   .71   4.4
 19  5.2     79  41    .6   .76   3.4,
      산도   유효인산  유기물  마그네슘    칼륨    칼슘
 0   5.5     30   81    .5   .48    .5
 1   5.5     25   75    .3   .56    .3
 2   5.5     22   67    .3   .44    .2
 3   5.5     19   69    .3   .47    .1
 4   5

In [86]:
dfs[2]

,산도,유효인산,유기물,마그네슘,칼륨,칼슘
0,5.5,30,81,.5,.48,.5
1,5.5,25,75,.3,.56,.3
2,5.5,22,67,.3,.44,.2
3,5.5,19,69,.3,.47,.1
4,5.5,25,60,.2,.54,.2
5,5.5,27,60,.2,.52,.1
6,5.5,25,61,.2,.49,.1
7,5.5,16,61,.2,.41,.1
8,5.5,30,62,.2,.58,.1
9,5.5,41,54,.2,.43,.1


In [ ]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
from urllib.parse import quote
with open('road_key_copy.txt') as file: # (필) 자생식물데이터 폴더에서 벗어나면 key 변경
    road_key = f'{file.read()}'

In [ ]:
df = pd.read_csv('자생식물1.csv')
df

,Unnamed: 0,이름,학명,주소
0,0,까마귀쪽나무,Litsea japonica (Thunb.) Juss.,한라수목원
1,1,좁은잎천선과,Ficus erecta var. sieboldii (Miq.) King,제주시 오등동
2,2,참식나무,Neolitsea sericea (Blume) Koidz.,제주특별자치도 제주시 조천읍
3,3,메밀,Fagopyrum esculentum Moench,서귀포시 안덕면 서광리
4,4,구실잣밤나무,Castanopsis sieboldii (Makino) Hatus. ex T. Ya...,만장굴
5,5,순비기나무,Vitex rotundifolia L. f.,제주특별자치도 제주시 삼양동
6,6,순비기나무,Vitex rotundifolia L. f.,제주특별자치도 제주시 구좌읍 월정리
7,7,참가시나무,Quercus salicina Blume,한라수목원
8,8,황근,Hibiscus hamabo Siebold & Zucc.,한라수목원
9,9,황근,Hibiscus hamabo Siebold & Zucc.,한라수목원


In [ ]:
adds = df['주소']
adds

0       서귀포시 안덕면 서광리
1             농산물원종장
2    제주특별자치도 제주시 삼양동
3         한라생태숲 목련총림
4              한라수목원
5           첨단입구 교차로
6                선덕사
Name: 주소, dtype: object

In [ ]:
adds = df['주소']
for i, add in enumerate(adds):

In [ ]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote(input())}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
result = requests.get(url).json()

BJD_code=result['results']['juso'][0]['admCd'] 
BJD_code

In [ ]:
addrr = pd.read_csv('전국.csv', sep=',')  #전국.csv만들기
addrr = pd.DataFrame(addrr)
addrr = addrr[['EMNDN_CD','CLZN_CD']]
addrr.columns = ['읍면동코드','기후대코드']
addrr['읍면동코드'] = int(BJD_code[:-2])
climate_code = addrr['기후대코드'].mode()[0]
climate_code  

# 1) 온대북부, 2)온대중부, 3)온대남부 4) 난대

# if climate_code == 1.0:
#     print('온대북부')
# elif climate_code == 2.0:
#     print('온대중부')
# elif climate_code == 3.0:
#     print('온대남부')
# elif climate_code == 4.0:
#     print('난대')